In [1]:
!pip install boto3
!pip install sagemaker

     |████████████████████████████████| 131 kB 7.7 MB/s            
     |████████████████████████████████| 79 kB 3.5 MB/s             
     |████████████████████████████████| 8.2 MB 71.4 MB/s            
     |████████████████████████████████| 452 kB 6.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 57 kB 3.1 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 81 kB 6.5 MB/s             
     |████████████████████████████████| 65 kB 2.0 MB/s             
     |████████████████████████████████| 128 kB 35.3 MB/s            
  Created wheel for sagemaker: filename=sagemaker-2.69.0-py2.py3-none-any.whl size=625707 sha256=ab8087503eb2bb953f06a84564d4d80ec4d13e68c07a643388aff57f29a7c782
  Stored in directory: /home/jovyan/.cache/pip/wheels/d7/8f/9d/2311179404dd882feb736a88de9cb56e4377445273ed4a7314
  Created wheel for protobuf3-to-dict: filename=protobuf3_to_dict-0.1.5-py3-none-any.whl size=40

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/players.csv')
df.head()

,PLAYER_NAME,SEASON_ID,PLAYER_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,SALARY_NOMINAL,SALARY_REAL
0,A.C. Green,1990-91,920,0,1610612747,LAL,27.0,82,21.0,2164.0,...,315.0,516.0,71,59.0,23.0,99.0,117,750,1750000.0,3473015.0
1,A.C. Green,1991-92,920,0,1610612747,LAL,28.0,82,53.0,2902.0,...,456.0,762.0,117,91.0,36.0,111.0,141,1116,1750000.0,3317240.0
2,A.C. Green,1992-93,920,0,1610612747,LAL,29.0,82,55.0,2819.0,...,424.0,711.0,116,88.0,39.0,116.0,149,1051,1750000.0,3217865.0
3,A.C. Green,1993-94,920,0,1610612756,PHX,30.0,82,55.0,2825.0,...,478.0,753.0,137,70.0,38.0,100.0,142,1204,1885000.0,3365286.0
4,A.C. Green,1994-95,920,0,1610612756,PHX,31.0,82,52.0,2687.0,...,475.0,669.0,127,55.0,31.0,114.0,146,916,6472600.0,11274438.0


In [20]:
df = df[['SEASON_ID', 'PLAYER_AGE', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'SALARY_NOMINAL']]
df.head(3)

,SEASON_ID,PLAYER_AGE,REB,AST,STL,BLK,PTS,SALARY_NOMINAL
0,1990-91,27.0,516.0,71,59.0,23.0,750,1750000.0
1,1991-92,28.0,762.0,117,91.0,36.0,1116,1750000.0
2,1992-93,29.0,711.0,116,88.0,39.0,1051,1750000.0


In [31]:
seasons_keys = df['SEASON_ID'].unique()
seasons_keys

array(['1990-91', '1991-92', '1992-93', '1993-94', '1994-95', '1995-96',
       '1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2016-17',
       '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15',
       '2007-08', '2008-09', '2015-16', '2017-18', '2018-19', '2019-20',
       '2020-21', '2021-22', '2001-02', '2002-03', '2003-04', '2004-05',
       '2005-06', '2006-07'], dtype=object)

In [65]:
seasons = { season: { 'df': df[df['SEASON_ID'] == season].drop(columns='SEASON_ID') } for season in seasons_keys }

In [66]:
seasons['2019-20']

{'df':        PLAYER_AGE    REB  AST   STL   BLK   PTS  SALARY_NOMINAL
 48           24.0  475.0  228  51.0  39.0   894      19863636.0
 68           23.0  156.0  225  55.0  16.0   627       2239200.0
 115          26.0  100.0   38  23.0  20.0   345       1618520.0
 145          21.0   10.0    4   4.0   0.0    32         79568.0
 152          23.0   47.0   15   8.0   4.0    99       1000000.0
 ...           ...    ...  ...   ...   ...   ...             ...
 16240        22.0   69.0   16   5.0   5.0    77       4240200.0
 16246        25.0  289.0  254  88.0  28.0  1530      19500000.0
 16315        21.0    2.0    2   3.0   0.0     8       3058800.0
 16319        19.0  150.0   50  16.0   9.0   540       9757440.0
 16340        24.0    9.0    3   1.0   1.0    12         79568.0
 
 [557 rows x 7 columns]}

In [67]:
from sklearn.model_selection import train_test_split

for season, data in seasons.items():
    X = data['df'].drop(columns='SALARY_NOMINAL')
    y = data['df']['SALARY_NOMINAL']
    data['X_train'], data['X_test'], data['y_train'], data['y_test'] = train_test_split(X, y)

In [68]:
print(seasons['2019-20']['X_train'].shape, seasons['2019-20']['y_train'].shape)
print(seasons['2019-20']['X_test'].shape, seasons['2019-20']['y_test'].shape)

(417, 6) (417,)
(140, 6) (140,)


In [72]:
from sklearn.linear_model import LinearRegression

for season, data in seasons.items():
    model = LinearRegression()
    model.fit(data['X_train'], data['y_train'])
    data['model'] = model

In [77]:
seasons['2019-20']['model'].coef_

array([623753.71136291,   5737.52586975,  16121.83791182, -41393.72120075,
        35684.51890822,   7458.72699727])

In [86]:
seasons['2019-20']['model'].feature_names_in_

array(['PLAYER_AGE', 'REB', 'AST', 'STL', 'BLK', 'PTS'], dtype=object)

In [81]:
pred = seasons['2019-20']['model'].predict(seasons['2019-20']['X_test'])
# pred

In [95]:
seasons['2020-21']['X_test']

,PLAYER_AGE,REB,AST,STL,BLK,PTS
12087,27.0,405.0,250,64.0,37.0,1196
8710,26.0,77.0,52,15.0,2.0,245
13394,23.0,312.0,82,44.0,7.0,789
9377,23.0,24.0,8,5.0,8.0,57
11647,22.0,1.0,1,0.0,1.0,9
...,...,...,...,...,...,...
6526,23.0,85.0,15,11.0,20.0,104
13010,30.0,466.0,117,101.0,84.0,593
13421,32.0,750.0,763,89.0,23.0,1445
5695,26.0,47.0,32,17.0,4.0,184


In [107]:
import joblib

models = { season: data['model'] for season, data in seasons.items() }
models
# joblib.dump(models, '../data/models.joblib')

{'1990-91': LinearRegression(),
 '1991-92': LinearRegression(),
 '1992-93': LinearRegression(),
 '1993-94': LinearRegression(),
 '1994-95': LinearRegression(),
 '1995-96': LinearRegression(),
 '1996-97': LinearRegression(),
 '1997-98': LinearRegression(),
 '1998-99': LinearRegression(),
 '1999-00': LinearRegression(),
 '2000-01': LinearRegression(),
 '2016-17': LinearRegression(),
 '2009-10': LinearRegression(),
 '2010-11': LinearRegression(),
 '2011-12': LinearRegression(),
 '2012-13': LinearRegression(),
 '2013-14': LinearRegression(),
 '2014-15': LinearRegression(),
 '2007-08': LinearRegression(),
 '2008-09': LinearRegression(),
 '2015-16': LinearRegression(),
 '2017-18': LinearRegression(),
 '2018-19': LinearRegression(),
 '2019-20': LinearRegression(),
 '2020-21': LinearRegression(),
 '2021-22': LinearRegression(),
 '2001-02': LinearRegression(),
 '2002-03': LinearRegression(),
 '2003-04': LinearRegression(),
 '2004-05': LinearRegression(),
 '2005-06': LinearRegression(),
 '2006-0

In [100]:
payload = [
    {'SEASON_ID': '2019-20', 'PLAYER_AGE': 27, 'REB': 321, 'AST': 50, 'STL': 23, 'BLK': 48, 'PTS': 438},
    {'SEASON_ID': '2020-21', 'PLAYER_AGE': 27, 'REB': 405, 'AST': 250, 'STL': 64, 'BLK': 37, 'PTS': 1196},
    {'SEASON_ID': '2020-21', 'PLAYER_AGE': 26, 'REB': 77, 'AST': 52, 'STL': 15, 'BLK': 2, 'PTS': 245, 'BLA': 123}
]
payload

[{'SEASON_ID': '2019-20',
  'PLAYER_AGE': 27,
  'REB': 321,
  'AST': 50,
  'STL': 23,
  'BLK': 48,
  'PTS': 438},
 {'SEASON_ID': '2020-21',
  'PLAYER_AGE': 27,
  'REB': 405,
  'AST': 250,
  'STL': 64,
  'BLK': 37,
  'PTS': 1196},
 {'SEASON_ID': '2020-21',
  'PLAYER_AGE': 26,
  'REB': 77,
  'AST': 52,
  'STL': 15,
  'BLK': 2,
  'PTS': 245,
  'BLA': 123}]

In [117]:
features = seasons['2019-20']['model'].feature_names_in_
features

array(['PLAYER_AGE', 'REB', 'AST', 'STL', 'BLK', 'PTS'], dtype=object)

In [118]:
data = pd.json_normalize(payload).fillna(0)

In [119]:
data

,SEASON_ID,PLAYER_AGE,REB,AST,STL,BLK,PTS,BLA
0,2019-20,27,321,50,23,48,438,0.0
1,2020-21,27,405,250,64,37,1196,0.0
2,2020-21,26,77,52,15,2,245,123.0


In [120]:
seasons_keys = data['SEASON_ID'].unique()
seasons_keys

array(['2019-20', '2020-21'], dtype=object)

In [121]:
import numpy as np
teste = []
for season in seasons_keys:
    df = data[data['SEASON_ID'] == season]
    pred = models[season].predict(df[features])
    teste = np.append(teste, pred)
list(teste)

[8807224.466083797, 17676268.24168354, 4834905.879901029]

In [127]:
['asd', 'asdf'] + [6]

['asd', 'asdf', 6]

In [171]:
import pandas as pd
import pickle
file = open('../data/players_stats.pkl', 'rb')
players = pickle.load(file)
file.close()

In [179]:
players[0]['info'].common_player_info.get_data_frame()['DISPLAY_FIRST_LAST'][0]

'Alaa Abdelnaby'

In [187]:
ret = []
for player in players:
    p = {
        'id': player['id'],
        'name': players[0]['info'].common_player_info.get_data_frame()['DISPLAY_FIRST_LAST'][0],
        'seasons': player['career'].season_totals_regular_season.get_data_frame()['SEASON_ID'].values.tolist()
    }
    ret.append(p)

In [188]:
ret

[{'id': 76001,
  'name': 'Alaa Abdelnaby',
  'seasons': ['1990-91',
   '1991-92',
   '1992-93',
   '1992-93',
   '1992-93',
   '1993-94',
   '1994-95',
   '1994-95',
   '1994-95']},
 {'id': 76002,
  'name': 'Alaa Abdelnaby',
  'seasons': ['1968-69',
   '1968-69',
   '1968-69',
   '1969-70',
   '1970-71',
   '1971-72',
   '1972-73',
   '1973-74',
   '1974-75',
   '1975-76',
   '1976-77',
   '1977-78',
   '1977-78',
   '1977-78']},
 {'id': 76003,
  'name': 'Alaa Abdelnaby',
  'seasons': ['1969-70',
   '1970-71',
   '1971-72',
   '1972-73',
   '1973-74',
   '1974-75',
   '1975-76',
   '1976-77',
   '1977-78',
   '1978-79',
   '1979-80',
   '1980-81',
   '1981-82',
   '1982-83',
   '1983-84',
   '1984-85',
   '1985-86',
   '1986-87',
   '1987-88',
   '1988-89']},
 {'id': 51,
  'name': 'Alaa Abdelnaby',
  'seasons': ['1990-91',
   '1991-92',
   '1992-93',
   '1993-94',
   '1994-95',
   '1995-96',
   '1996-97',
   '1997-98',
   '2000-01']},
 {'id': 1505,
  'name': 'Alaa Abdelnaby',
  'season